In [65]:
import pandas as pd
import requests
import json
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import config
import time

In [13]:
url = "https://raw.githubusercontent.com/Flores-Kevin/rage-against-machine-learning/molly/resources/regional-usa-weekly-2022-10-06.csv"

In [14]:
df = pd.read_csv(url)

In [15]:
df.head(1)

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams
0,1,spotify:track:3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),EMI,1,1,2,12004993


In [16]:
df.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams'],
      dtype='object')

In [17]:
#extracting track id (removing "spotify:track:" from track id in csv)
track_id_series = df.uri.str[14:]

In [18]:
#convert to df and rename column
track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})

In [19]:
#merge dfs
df = df.merge(track_id_df, how="inner", left_index=True, right_index=True)
df = df[["rank", "track_id", "artist_names", "track_name", "peak_rank", "previous_rank", "weeks_on_chart", "streams"]]

In [20]:
df

,rank,track_id,artist_names,track_name,peak_rank,previous_rank,weeks_on_chart,streams
0,1,3nqQXoyQOWXiESFLlDF1hG,"Sam Smith, Kim Petras",Unholy (feat. Kim Petras),1,1,2,12004993
1,2,4k6Uh1HXdhtusDW5y8Gbvy,Steve Lacy,Bad Habit,1,2,14,10425163
2,3,1IHWl5LamUGEuP4ozKQSXZ,Bad Bunny,Tití Me Preguntó,3,7,22,6742213
3,4,4Dvkj6JhhA12EX05fT7y2e,Harry Styles,As It Was,1,3,27,6540607
4,5,1xK59OXxi2TAAAbmZK0kBL,d4vd,Romantic Homicide,5,5,9,6375518
...,...,...,...,...,...,...,...,...
195,196,3yfqSUWxFvZELEM4PmlwIR,Eminem,The Real Slim Shady,48,194,28,1615334
196,197,6UelLqGlWMcVH1E5c4H7lY,Harry Styles,Watermelon Sugar,4,178,143,1604146
197,198,48q0vSHcJdhK3IiXH8C5WJ,Arctic Monkeys,Why'd You Only Call Me When You're High?,111,-1,14,1603256
198,199,4MzXwWMhyBbmu6hOcLVD49,"Bad Bunny, Jhay Cortez",DÁKITI,1,186,92,1601541


In [21]:
first_track_id = df["track_id"][0]

In [22]:
#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [23]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': config.client_id,
    'client_secret': config.client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [24]:
#base URL for Spotify API
base_url = 'https://api.spotify.com/v1/'

track_response = requests.get(base_url + 'audio-features/' + first_track_id, headers=headers).json()

In [25]:
#empty lists to store song data
global_danceability = []
global_energy = []
global_key = []
global_loudness = []
global_mode = []
global_speechiness = []
global_acousticness = []
global_instrumentalness = []
global_liveness = []
global_valence = []
global_tempo = []
global_id_num = []
global_track_href = []
global_analysis_url = []
global_duration_mins = []
global_time_signature = []

In [28]:
#loop through dataframe, use track id to look up songs in spotify api and grab each one's audio features
for t in df["track_id"]:
    global_response = requests.get(base_url + 'audio-features/' + t, headers=headers).json()
    
    time.sleep(1)
    
    #fill in track info for each audio feature / empty list
    global_danceability.append(global_response["danceability"])
    global_energy.append(global_response["energy"])
    global_key.append(global_response["key"])
    global_loudness.append(global_response["loudness"])
    global_mode.append(global_response["mode"])
    global_speechiness.append(global_response["speechiness"])
    global_acousticness.append(global_response["acousticness"])
    global_instrumentalness.append(global_response["instrumentalness"])
    global_liveness.append(global_response["liveness"])
    global_valence.append(global_response["valence"])
    global_tempo.append(global_response["tempo"])
    global_id_num.append(global_response["id"])
    global_track_href.append(global_response["track_href"])
    global_analysis_url.append(global_response["analysis_url"])    
    #duration recorded in api in milliseconds, converting to minutes here
    global_duration_mins.append(round((global_response["duration_ms"] / 60000),2))
    global_time_signature.append(global_response["time_signature"])

In [63]:
print(global_danceability)

[0.714, 0.686, 0.65, 0.52, 0.571, 0.911, 0.951, 0.369, 0.561, 0.529, 0.704, 0.697, 0.733, 0.586, 0.612, 0.603, 0.44, 0.511, 0.801, 0.731, 0.463, 0.801, 0.69, 0.78, 0.714, 0.335, 0.733, 0.761, 0.367, 0.637, 0.804, 0.608, 0.836, 0.52, 0.768, 0.346, 0.647, 0.505, 0.723, 0.876, 0.582, 0.629, 0.818, 0.591, 0.79, 0.9, 0.902, 0.711, 0.554, 0.467, 0.369, 0.594, 0.607, 0.698, 0.621, 0.829, 0.639, 0.716, 0.66, 0.835, 0.791, 0.566, 0.647, 0.849, 0.676, 0.564, 0.614, 0.723, 0.715, 0.485, 0.87, 0.558, 0.575, 0.323, 0.645, 0.228, 0.625, 0.585, 0.795, 0.679, 0.754, 0.578, 0.572, 0.49, 0.663, 0.772, 0.352, 0.495, 0.645, 0.659, 0.877, 0.914, 0.515, 0.514, 0.461, 0.721, 0.601, 0.755, 0.664, 0.881, 0.897, 0.464, 0.743, 0.773, 0.773, 0.68, 0.913, 0.801, 0.693, 0.671, 0.883, 0.762, 0.741, 0.963, 0.565, 0.563, 0.746, 0.511, 0.545, 0.508, 0.724, 0.332, 0.677, 0.392, 0.615, 0.631, 0.529, 0.789, 0.8, 0.627, 0.413, 0.695, 0.371, 0.442, 0.824, 0.372, 0.602, 0.828, 0.588, 0.544, 0.559, 0.794, 0.631, 0.579, 0.798,

In [32]:
# global_danceabilityctionary to hold data gathered from the api
global_songs_dict = {
    "track_id": global_id_num,
    "danceability": global_danceability,
    "energy": global_energy,
    "key": global_key,
    "loudness": global_loudness,
    "mode": global_mode,
    "speechiness": global_speechiness,
    "acousticness": global_acousticness,
    "instrumentalness": global_instrumentalness,
    "liveness": global_liveness,
    "valence": global_valence,
    "tempo": global_tempo,
    "duration_mins": global_duration_mins,
    "time_signature": global_time_signature,
    "track_href": global_track_href,
    "analysis_url": global_analysis_url
}

In [64]:
#convert dictionary to a dataframe
global_info_df = pd.DataFrame(global_songs_dict)
global_info_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature,track_href,analysis_url
0,3nqQXoyQOWXiESFLlDF1hG,0.714,0.472,2,-7.375,1,0.0864,0.0130,0.000005,0.266,0.238,131.121,2.62,4,https://api.spotify.com/v1/tracks/3nqQXoyQOWXi...,https://api.spotify.com/v1/audio-analysis/3nqQ...
1,4k6Uh1HXdhtusDW5y8Gbvy,0.686,0.494,1,-7.093,1,0.0355,0.6130,0.000058,0.402,0.700,168.946,3.87,4,https://api.spotify.com/v1/tracks/4k6Uh1HXdhtu...,https://api.spotify.com/v1/audio-analysis/4k6U...
2,1IHWl5LamUGEuP4ozKQSXZ,0.650,0.715,5,-5.198,0,0.2530,0.0993,0.000291,0.126,0.187,106.672,4.06,4,https://api.spotify.com/v1/tracks/1IHWl5LamUGE...,https://api.spotify.com/v1/audio-analysis/1IHW...
3,4Dvkj6JhhA12EX05fT7y2e,0.520,0.731,6,-5.338,0,0.0557,0.3420,0.001010,0.311,0.662,173.930,2.79,4,https://api.spotify.com/v1/tracks/4Dvkj6JhhA12...,https://api.spotify.com/v1/audio-analysis/4Dvk...
4,1xK59OXxi2TAAAbmZK0kBL,0.571,0.544,6,-10.613,1,0.0299,0.4530,0.008050,0.322,0.216,132.052,2.21,4,https://api.spotify.com/v1/tracks/1xK59OXxi2TA...,https://api.spotify.com/v1/audio-analysis/1xK5...


In [39]:
gloabl_info_df = gloabl_info_df.drop(columns={'track_id','track_href','analysis_url'})

In [40]:
#Scale the data
scaler = StandardScaler()
df_scaled = scaler.fit_transform(gloabl_info_df)

In [41]:
#Reducing dimensionality with pca
pca = PCA(n_components = 0.9)
df_pca = pca.fit_transform(df_scaled)

In [42]:
#Run t-SNE on the pca output to further reduce dimensionality.
tsne = TSNE()
df_tsne = tsne.fit_transform(df_pca)

/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/Users/kevin/opt/anaconda3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [62]:
#Plotting the tsne numpy array
plt.scatter(df_tsne[:,0],df_tsne[:,1],c=danceability)

NameError: name 'danceability' is not defined

In [45]:
# from sklearn.ensemble.forest import RandomForestRegressor

# # Can combine step above with this
# forest = RandomForestRegressor(random_state=42, max_depth=5, max_features=10) 
# forest.fit(X,Y)
# importances = forest.feature_importances_
# indices = np.argsort(importances)[::-1]

# # Print the feature rankings
# print("Playlist ranking:")
  
# for f in range(len(importances)):
#     print("%d. %s %f " % (f + 1, 
#             X.columns[f], 
#             importances[indices[f]]))

In [55]:
search_track = input("Please name a song: ")

Please name a song:  Yesterday


In [56]:
search_artist = input("Please name that song's artist: ")

Please name that song's artist:  The Beatles


In [57]:
response = requests.get(search_url + '?q=track%3A' + search_track + '%20artist%3A' + search_artist + '&type=track%2Cartist&limit=1', headers=headers).json()

In [59]:
response

{'error': {'status': 401, 'message': 'The access token expired'}}